<a href="https://colab.research.google.com/github/estherkxy/GA_Projects/blob/main/Capstone/code/6_Bert_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT Test

Notebook 6/6

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install tweet-preprocessor
import transformers
#from transformers import *
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
import tensorflow as tf

from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

from keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt
import seaborn as sns
import preprocessor as p
import pandas as pd
import numpy as np
import pandas as pd
import time
import datetime
import random
import io

%matplotlib inline  
print('Transformers version: ', transformers.__version__)
print('Tensorflow version: ', tf.__version__)

     |████████████████████████████████| 3.5 MB 8.2 MB/s 
     |████████████████████████████████| 596 kB 64.7 MB/s 
     |████████████████████████████████| 67 kB 7.4 MB/s 
     |████████████████████████████████| 895 kB 63.4 MB/s 
     |████████████████████████████████| 6.8 MB 56.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Transformers version:  4.16.2
Tensorflow version:  2.8.0


In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():     
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


# Importing Test Dataset

In [ ]:
test_df = pd.read_csv('./drive/My Drive/GA/capstone/data/tweets_test.csv')
test_df.head()

,id,text,sentiment
0,1,Wow. Yall needa step it up @Apple RT @heynyla:...,-1
1,2,What Happened To Apple Inc? http://t.co/FJEX...,0
2,3,Thank u @apple I can now compile all of the pi...,1
3,4,The oddly uplifting story of the Apple co-foun...,0
4,5,@apple can i exchange my iphone for a differen...,0


# Cleaning Dataset

In [ ]:
import regex as re
def tidy_up(df): #change the sentiment labels to binary and clean up tweets.
  df.sentiment = df.sentiment.map({0:0, 1:0, -1:1})
  df.drop(columns=['id'],inplace=True)
  df.drop_duplicates(subset='text',keep='last',inplace=True)
  df.reset_index(drop=True,inplace=True)
  p.set_options(p.OPT.URL, p.OPT.RESERVED, p.OPT.EMOJI, p.OPT.SMILEY)
  df['text']=[p.clean(x) for x in df['text']]
  df['text']=[re.sub(r'(&amp;)|(&lt;)|(b&gt;)|([!?.@#]*[!?.@#])| ([0-9])',"",x) for x in df['text']]


In [ ]:
tidy_up(test_df)
test_df['bert_predict'] = 0
test_df.sample(5)

,text,sentiment,bert_predict
17,Theyre not RT Naivana_: You gotta be kidding m...,1,0
1093,INSIGHT FROM APPLE'S TIME COOK tim_cook apple ...,0,0
93,SwiftKey Apple Oh In wish you guys would port ...,1,0
1146,Breyythatsmee: I have70 pictures and my storag...,1,0
824,"Steve Jobs, clave en juicio contra Apple",0,0


In [ ]:
# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(test_df.shape[0]))

# Display 5 random rows from the data.
test_df.sample(5, random_state=1)

Number of test sentences: 1,624



,text,sentiment,bert_predict
1276,"Torn between a googlenexus, samsungmobile Note...",0,0
60,adage: Watch the weekend's new TV ads from Rad...,0,0
1486,"Apple Wants To Make Your Commute Much Easier, ...",0,0
1166,stevewoz: passion for engineering/design at t...,0,0
993,Apple's iPhone6 International Prices Are Highe...,0,0


In [ ]:
test_df.sentiment.value_counts()

0    941
1    683
Name: sentiment, dtype: int64

In [ ]:
test_df.sentiment.value_counts(normalize = True)

0    0.579433
1    0.420567
Name: sentiment, dtype: float64

Test set is slightly imblanced (58% non-negatives vs 42% negatives)

# Loading BERT model

In [ ]:
output_dir = './drive/My Drive/GA/capstone/BERT with cleaned tweets' #loading cleaned tweets BERT

model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

# BERT Predictions

In [ ]:
test_df['bert_predict'] = 0

# Create sentence and label lists
tweets = test_df.text.values
labels = test_df.bert_predict.values

# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []

# For every sentence...
for tweet in tweets:
    encoded_tweet = tokenizer.encode( tweet, add_special_tokens = True )
    input_ids.append(encoded_tweet)

# Pad our input tokens
MAX_LEN = max([len(i) for i in input_ids])
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

# Predictions

print('Predicting labels for {:,} tweets...'.format(len(prediction_inputs)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions_clean , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions_clean.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 1,624 tweets...
    DONE.


In [ ]:
clean_flat_prediction = [item for sublist in predictions_clean for item in sublist]
clean_flat_predictions = np.argmax(clean_flat_prediction, axis=1).flatten()
test_df['bert_predict'] = clean_flat_predictions

In [ ]:
tn, fp, fn, tp = confusion_matrix(test_df['sentiment'], test_df['bert_predict']).ravel()
print("True Negatives: %s" % tn)
print("False Positives: %s" % fp)
print("False Negatives: %s" % fn)
print("True Positives: %s" % tp)

accuracy_clean = (tp + tn) / (tp + tn + fp + fn)
print(f'Accuracy: {round(accuracy_clean,4)}')

misclass_clean = (fp + fn) / (tp + tn + fp + fn)
print(f'Misclassification Rate: {round(misclass_clean,4)}')

spec_clean = tn / (tn + fp) #how many of the 'spam' was correctly classified
print(f'Specificity: {round(spec_clean,4)}')

sens_clean = tp / (tp + fn) #how many of the disasters was correctly identified
print(f'Sensitivity: {round(sens_clean,4)}')

True Negatives: 908
False Positives: 33
False Negatives: 33
True Positives: 650
Accuracy: 0.9594
Misclassification Rate: 0.0406
Specificity: 0.9649
Sensitivity: 0.9517


In [ ]:
test_df.head()

,text,sentiment,bert_predict
0,Wow Yall needa step it up Apple RT heynyla: Mu...,1,1
1,What Happened To Apple Inc aapl apple MoneyPress,0,0
2,Thank u apple I can now compile all of the pic...,0,0
3,The oddly uplifting story of the Apple co-foun...,0,0
4,apple can i exchange my iphone for a different...,0,0


In [ ]:
pd.set_option('display.max_colwidth', -1)
test_df.loc[test_df['bert_predict']!=test_df['sentiment']]['text'].sample().to_string()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


'114    appleipad irig For the price to connect my guitar I could buy a real amp loud enough to perform ripoff'

In [ ]:
test_df.to_csv('./drive/My Drive/GA/capstone/BERTtest_df.csv')

In [ ]:
test_df['check'] = test_df['sentiment'] - test_df['bert_predict']
test_df.head()

,text,sentiment,bert_predict,check
0,Wow Yall needa step it up Apple RT heynyla: Music and snapchat at the same damn time Thank you Note4,1,1,0
1,What Happened To Apple Inc aapl apple MoneyPress,0,0,0
2,Thank u apple I can now compile all of the pics that i communicate with in one place,0,0,0
3,The oddly uplifting story of the Apple co-founder who sold his stake for $800 $AAPL aapl,0,0,0
4,apple can i exchange my iphone for a different color lmao i changed my mind,0,0,0


In [ ]:
checking_df = test_df[test_df['check'] != 0]
checking_df.head()

,text,sentiment,bert_predict,check
10,Facebook CEO Mark Zuckerberg criticizes Apple $AAPL aapl,1,0,1
36,PCAudioLabs is in and Apple is out at Emanon Records Apple was premium product Now it is an average product with a premium price,1,0,1
65,Nasdaq big fall FB Baba Twtr Linkd Apple fell-6% on Monday See whether follow up,1,0,1
109,ohaiigloria hahahamark apple explain yourselves,1,0,1
114,appleipad irig For the price to connect my guitar I could buy a real amp loud enough to perform ripoff,1,0,1


From the tables above, we can see that BERT model struggles slightly with identifying the overall tone of tweets that have both positive and negative tones with the model showing a slight tendency to predict the tweet as negative rather than positive. This is seen in the slightly lower score in specificity (true negative rate) as compared to sensitivity (true positive rate). However, I would also note that the scores are all well above 95% and this indicates that the model has done relatively well in predicting the sentiment of tweets in general. 

In [ ]:
checking_df['check'].value_counts()

 1    33
-1    33
Name: check, dtype: int64

# Results Evaluation

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

# f1, precision, and recall scores
precision_score = precision_score(test_df['sentiment'], 
                                           test_df['bert_predict'])
recall_score = recall_score(test_df['sentiment'], test_df['bert_predict'])
f1 = f1_score(test_df['sentiment'], test_df['bert_predict'])


results_list = [accuracy_clean, precision_score, recall_score, f1, 
                        misclass_clean, spec_clean, sens_clean]
results_df = pd.DataFrame(results_list, index=['accuracy', 'precision', 
                                               'recall', 'f1',
                                               'misclassification', 
                                               'specificity', 'sensitivity']).T

results_df

,accuracy,precision,recall,f1,misclassification,specificity,sensitivity
0,0.95936,0.951684,0.951684,0.951684,0.04064,0.964931,0.951684


In [ ]:
results_df.to_csv('./drive/My Drive/GA/capstone/BERTscores.csv')

In [ ]:
# Loading train and test results
bert_train = pd.read_csv('./drive/My Drive/GA/capstone/data/bert_train_results_df.csv')
bert_val_scores = pd.read_csv('./drive/My Drive/GA/capstone/data/bert_train_scores.csv')
ml_train = pd.read_csv('./drive/My Drive/GA/capstone/data/MLmetrics_df.csv')
w2v_train = pd.read_csv('./drive/My Drive/GA/capstone/data/w2v_results_df.csv')
we_train = pd.read_csv('./drive/My Drive/GA/capstone/data/wordembeddings_results_df.csv')

In [ ]:
ml_train = ml_train.sort_values(by=['test', 'recall'], ascending=False).reset_index(drop=True).iloc[[0]]
ml_train.drop(['Unnamed: 0'], axis=1, inplace=True)
if ml_train['smote'].any():
  ml_train['model'] = ml_train['model'] + '-' + ml_train['vectorizer'] + '-smote'
  ml_train.drop(columns=['vectorizer', 'smote'], inplace=True)

else:
  ml_train['model'] = ml_train['model'] + '-' + ml_train['vectorizer'] + '-no_smote'
  ml_train.drop(columns=['vectorizer', 'smote'], inplace=True)
ml_train.set_index('model', inplace=True)
ml_train.drop(columns=['train','roc'], inplace=True)
ml_train.rename(columns={'test':'f1_score'}, inplace=True)

column_titles = ['train_acc',	'val_acc', 'f1_score',	'precision',	'recall']
ml_train = ml_train[column_titles]

ml_train

,train_acc,val_acc,f1_score,precision,recall
model,,,,,
cnb-cvec-smote,0.880621,0.786963,0.72541,0.657993,0.808219


In [ ]:
w2v_train1 = w2v_train.iloc[[-1]].drop(columns=['train_loss', 'val_loss'])
w2v_train1.rename(columns={'epoch':'model'}, inplace=True)
w2v_train1['model'] = 'word2vec'
w2v_train1.set_index('model', inplace=True)
we_train1 = we_train.iloc[[-1]].drop(columns=['train_loss', 'val_loss'])
we_train1.rename(columns={'epoch':'model'}, inplace=True)
we_train1['model'] = 'word_embeddings'
we_train1.set_index('model', inplace=True)
display(w2v_train1)
display(we_train1)

,train_acc,val_acc,f1_score,precision,recall
model,,,,,
word2vec,0.655131,0.671975,0.517252,0.535211,0.086659


,train_acc,val_acc,f1_score,precision,recall
model,,,,,
word_embeddings,0.899404,0.781847,0.530529,0.883939,0.830396


In [ ]:
bert_val_scores.drop(columns=['sensitivity', 'Unnamed: 0', 'specificity'], 
                     inplace=True)
bert_val_scores.rename(columns={'f1': 'f1_score', 'accuracy': 'val_acc'}, 
                       inplace=True)
bert_val_scores['model'] = 'BERT'
bert_val_scores['train_acc'] = 'NaN'
bert_val_scores.set_index('model', inplace=True)
column_titles = ['train_acc',	'val_acc', 'f1_score',	'precision',	'recall']
bert_val_scores = bert_val_scores[column_titles]
display(bert_val_scores)

,train_acc,val_acc,f1_score,precision,recall
model,,,,,
BERT,NaN,0.863275,1.314056,0.810573,0.810573


In [ ]:
combined_train = ml_train.append(w2v_train1).append(we_train1).append(bert_val_scores)
combined_train['misclassification'] = 1 - combined_train['val_acc']
combined_train

,train_acc,val_acc,f1_score,precision,recall,misclassification
model,,,,,,
cnb-cvec-smote,0.880621,0.786963,0.725410,0.657993,0.808219,0.213037
word2vec,0.655131,0.671975,0.517252,0.535211,0.086659,0.328025
word_embeddings,0.899404,0.781847,0.530529,0.883939,0.830396,0.218153
BERT,NaN,0.863275,1.314056,0.810573,0.810573,0.136725


In [ ]:
combined_train = combined_train.sort_values(by=['f1_score', 'recall'], 
                                            ascending=False).reset_index(
                                                drop=False)
combined_train

,model,train_acc,val_acc,f1_score,precision,recall,misclassification
0,BERT,NaN,0.863275,1.314056,0.810573,0.810573,0.136725
1,cnb-cvec-smote,0.880621,0.786963,0.725410,0.657993,0.808219,0.213037
2,word_embeddings,0.899404,0.781847,0.530529,0.883939,0.830396,0.218153
3,word2vec,0.655131,0.671975,0.517252,0.535211,0.086659,0.328025


In [ ]:
# Loading test results
bert_test = pd.read_csv('./drive/My Drive/GA/capstone/BERTscores.csv')
ml_test = pd.read_csv('./drive/My Drive/GA/capstone/data/MLtest_results.csv')
w2v_test = pd.read_csv('./drive/My Drive/GA/capstone/data/w2v_results_test.csv')
we_test = pd.read_csv('./drive/My Drive/GA/capstone/data/we_results_test.csv')

In [ ]:
ml_test.drop(['Unnamed: 0'], axis=1, inplace=True)
w2v_test.drop(['Unnamed: 0', 'sensitivity'], axis=1, inplace=True)
we_test.drop(['Unnamed: 0', 'sensitivity'], axis=1, inplace=True)
bert_test.drop(['sensitivity'], axis=1, inplace=True)

In [ ]:
display(bert_test)
display(ml_test)
display(w2v_test)
display(we_test)

,Unnamed: 0,accuracy,precision,recall,f1,misclassification,specificity
0,0,0.95936,0.951684,0.951684,0.951684,0.04064,0.964931


,model,vectorizer,train,test,train_acc,val_acc,roc,precision,recall,smote
0,cnb,cvec,0.838008,0.894309,0.87683,0.904294,0.912194,0.835443,0.962099,1


,accuracy,precision,recall,f1,misclassification,specificity
0,0.627607,0.780142,0.16035,0.266022,0.372393,0.967161


,accuracy,precision,recall,f1,misclassification,specificity
0,0.714724,0.991111,0.325073,0.489572,0.285276,0.997881


In [ ]:
if ml_test['smote'].any():
  ml_test['model'] = ml_test['model'] + '-' + ml_test['vectorizer'] + '-smote'

else:
  ml_test['model'] = ml_test['model'] + '-' + ml_test['vectorizer'] + '-no_smote'

ml_test.drop(columns=['train','train_acc','roc', 'vectorizer', 'smote'], 
             inplace=True)
ml_test.rename(columns={'test':'f1', 'val_acc':'accuracy'}, inplace=True)
ml_test['misclassification'] = 1 - ml_test['accuracy']

column_titles = ['model', 'accuracy',	'precision',	'recall', 'f1', 
                 'misclassification']
ml_test = ml_test[column_titles]
ml_test

,model,accuracy,precision,recall,f1,misclassification
0,cnb-cvec-smote,0.904294,0.835443,0.962099,0.894309,0.095706


In [ ]:
bert_test['model'] = 'BERT'
w2v_test['model'] = 'w2v'
we_test['model'] = 'word_embedding'

In [ ]:
combined = w2v_test.append(we_test).append(bert_test).append(ml_test)
combined = combined.sort_values(by=['f1', 'recall'],
                                ascending=False).reset_index(drop=True)
combined.drop(columns=['Unnamed: 0'], inplace=True)
combined.set_index('model', inplace=True)
combined

,accuracy,precision,recall,f1,misclassification,specificity
model,,,,,,
BERT,0.959360,0.951684,0.951684,0.951684,0.040640,0.964931
cnb-cvec-smote,0.904294,0.835443,0.962099,0.894309,0.095706,NaN
word_embedding,0.714724,0.991111,0.325073,0.489572,0.285276,0.997881
w2v,0.627607,0.780142,0.160350,0.266022,0.372393,0.967161


In [ ]:
# Exporting results
combined_train.to_csv('./drive/My Drive/GA/capstone/data/combinedtrain_results.csv')
combined.to_csv('./drive/My Drive/GA/capstone/data/combinedtest_results.csv')

# Live Testing

In [ ]:
def live_test(run=True):
  if run:
    # Creating user input 
    tweettext = input("Enter a tweet: ")
    testing = {'text': [tweettext]}
    # Create DataFrame with prediction column
    testing_df = pd.DataFrame(testing)
    testing_df['bert_predict'] = 0

    # Create sentence and label lists
    tweets = testing_df.text.values
    labels = testing_df.bert_predict.values

    # Tokenize all of the sentences and map the tokens to their word IDs.
    input_ids = []

    # For every sentence...
    for tweet in tweets:
      encoded_tweet = tokenizer.encode( tweet, add_special_tokens = True )
      input_ids.append(encoded_tweet)

    # Pad our input tokens
    MAX_LEN = max([len(i) for i in input_ids])
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # Create attention masks
    attention_masks = []

    # Create a mask of 1s for each token followed by 0s for padding
    for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask) 

    # Convert to tensors.
    prediction_inputs = torch.tensor(input_ids)
    prediction_masks = torch.tensor(attention_masks)
    prediction_labels = torch.tensor(labels)

    # Set the batch size.  
    batch_size = 32  

    # Create the DataLoader.
    prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
    prediction_sampler = SequentialSampler(prediction_data)
    prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

    # Predictions

    print('Predicting labels for {:,} tweets...'.format(len(prediction_inputs)))

    # Put model in evaluation mode
    model.eval()

    # Tracking variables 
    predictions_clean , true_labels = [], []

    # Predict 
    for batch in prediction_dataloader:
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
  
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch
  
      # Telling the model not to compute or store gradients, saving memory and 
      # speeding up prediction
      with torch.no_grad():
          # Forward pass, calculate logit predictions
          outputs = model(b_input_ids, token_type_ids=None, 
                          attention_mask=b_input_mask)

      logits = outputs[0]

      # Move logits and labels to CPU
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()
  
      # Store predictions and true labels
      predictions_clean.append(logits)
      true_labels.append(label_ids)

    print('    DONE.')
    clean_flat_prediction = [item for sublist in predictions_clean for item in sublist]
    clean_flat_predictions = np.argmax(clean_flat_prediction, axis=1).flatten()
    testing_df['bert_predict'] = clean_flat_predictions

    if testing_df['bert_predict'][0] == 1:
      print('Tweet is negative')
    else:
      print('Tweet is not negative')

  else:
   pass

In [ ]:
live_test(run=True)

Enter a tweet: iphones are great
Predicting labels for 1 tweets...
    DONE.
Tweet is not negative


In [ ]:
live_test(run=True)

Enter a tweet: iphone battery life sucks
Predicting labels for 1 tweets...
    DONE.
Tweet is negative


In [ ]:
live_test(run=True)

Enter a tweet: love the iphone, hate the airpods
Predicting labels for 1 tweets...
    DONE.
Tweet is negative


In [ ]:
live_test(run=True)

Enter a tweet: love the iphone, airpods are not great
Predicting labels for 1 tweets...
    DONE.
Tweet is not negative


In [ ]:
live_test(run=True)

Enter a tweet: love the iphone, airpods are okay
Predicting labels for 1 tweets...
    DONE.
Tweet is not negative


In [ ]:
live_test(run=True)

Enter a tweet: steve jobs
Predicting labels for 1 tweets...
    DONE.
Tweet is not negative
